## 2. Unsupervised Learning in the Ising Model
In this problem you will use principal component analysis (PCA) to identify phases in the Ising
model without any explicit labels. You will reproduce Figures 1 and 2 of L. Wang, Phys. Rev. B 94,
195105 (2016), https://arxiv.org/abs/1606.00318.

As in problem 1, the data is stored in an $M\times N$ matrix $X$, where each of the $M$ rows stores a spin configuration (a set of integers $\pm1$ corresponding to up/down) for a system with $N = L^2$ spins. The data files corresponding to $L = 20, 40, 80$ are included in the data directory of the course repository as compressed files: Ising2D_config_LZ.dat.gz where $Z=20,40,80$.

Each file contains 100 spin configurations at each of the 20 temperatures $T/ J = 1.0, 1.1, 1.2, . . . , 2.9$ such that $M = 2000$ for each lattice size.

For each $L$, there is a corresponding file storing the temperature at which the configuration was generated named Ising2D_temps_LZ.dat where $Z = 20,40,80$.

(a) Read in the spin configurations for the Ising model for each lattice size and determines the principal components $\mathbf{v}_j$. Make a scatter plot of the first two projected principal components $\mathbf{x'}_1=\mathbf{X}\mathbf{v}_1$ vs. $\mathbf{x'}_2=\mathbf{X}\mathbf{v}_2$ for each lattice size. Do you observe any trends as L is increased?

(b) Label the points in your plot such that they are coloured according to their temperature and
compare with Figure 2 of the Wang reference. Can you distinguish between the phases of the
2D Ising model?

(c) Consider now the explained variance ratios

$r_l=\frac{\lambda_l}{\sum_{i=1}^N \lambda_i}$

Plot the largest 10 values of $r_l$ for each lattice size and compare with Figure 1 of the reference.
How many principal components are needed to explain how the Ising spin configurations vary
as a function of temperature?
